## Predicting Airlines Stocks based on Oil Prices

#### 1. Include all libraries

In [ ]:
library(sqldf)

#### 2. Read in all datasets

In [ ]:
delta = read.csv('/Users/birupakhya/Documents/Projects/SDMproject/Deltastocks.csv', skip = 0, header = T, as.is = T, na.strings = "NA")
jetfuelprices = read.csv('/Users/birupakhya/Documents/Projects/SDMproject/jetfuelprices.csv', skip = 12, header = T, as.is = T, na.strings = "")
brentprices = read.csv('/Users/birupakhya/Documents/Projects/SDMproject/DCOILBRENTEU.csv',skip = 0, header = T, as.is = T, na.strings = ".")

#### 3. Check Missing values

In [ ]:
#check missing values
colSums(is.na(delta))
colSums(is.na(jetfuelprices))
colSums(is.na(brentprices))